# Predicción de Calidad de Café

**Objetivo**  
Predecir la calidad del café basados en el Puntaje de Taza.

---

### 1. Análisis de datos y selección de variables de entrada (X)
- Cargar y explorar el conjunto de datos.  
- Definir las variables predictoras (**X**), cuidando que no se filtre información del target.

### 2. Preprocesamiento
- **X** (features):
  - Imputación de valores faltantes  
  - Normalización / estandarización  
  - Codificación de variables categóricas  
- **Y** (Puntaje de Taza):
  - Revisión de distribución  
  - Transformaciones si aplica (e.g., escalado, discretización)

### 3. Entrenamiento de modelos de regresión
- Modelo 1: _Linear Regression_ (u otro algoritmo lineal)  
- Modelo 2: _Random Forest Regressor_ (u otro algoritmo no lineal)  

### 4. Explicabilidad
- Aplicar técnicas de interpretabilidad:
  - LIME  
  - SHAP  
  - Importancia de características (Feature Importance)

### 5. Reporte de resultados
- Métricas de evaluación: RMSE, MAE, R²  
- Comparación de desempeño entre modelos  
- Visualizaciones e interpretaciones  
